In [ ]:
%load_ext autoreload
%autoreload 2

# Sonecule: MultivariateBasicAUD – A basic playbuf driven Audification of n-dimensional data

This notebook introduces and demonstrates usage of the `MultivariateBasicAUD` sonecule.
- The sonecule enables a simple Audification of a data frame with $n$ channels.
- It can be initialized with 
  - a pya Asig (i.e. audio signal using pya)
    - `MultivariateBasicAUD(asig, sr=None, channels=None)`
  - a pandas DataFrame or Series
    - `MultivariateBasicAUD.from_df(df, sr=44100, time_column=None, columns=None)`
  - a numpy ndarray
    - `.from_np(data, sr=44100, time_column=None, columns=None)`
- Preprocessing such as time stretching, slicing, filtering is offered by specialized functions, either in pya (iirfilter, stretch) or libraries such as scipy.signal - correspondingly processed signals can be passed into Audification modules for audition and interaction.
- The current MultivariateBasicAUD uses SuperCollider3, controlled via sc3nb as Backend and therein uses a PlayBuf UGen for audification, which allows one-shot, looped, or repeatedly triggered playback of a data buffer.
- `MultivariateBasicAUD` is a *composite* sonecule, i.e., it wraps $n$ BasicAUD sonecules and orchestrates their controls.
- The synth is mutable, i.e., its parameters can be controlled interactively (code or GUI)
  - should enable pause/resume
  - with rate control (note: not band-limited!): a number, shared for all channels
  - with amp control: a number or array/list (then amplitude for individual channels)
  - with pan control: a number or array/list (then panning for individual channels)
  - with onset (i.e. at in schedule)  

Let's get started. First some imports and settings and startup of sonecules

In [ ]:
# headers and imports for the demo
import sonecules as sn
from pya import Asig
import pyamapping as pam
import matplotlib.pyplot as plt

# setup for matplotlib 
plt.rcParams["figure.figsize"] = (8,3)
%matplotlib widget

# start sonecules (with default backend sc3nb, aka sc3)
sn.startup()
ctx = sn.gcc()  # get the context as ctx

Load data sets used for the demo

In [ ]:
%run ../data/prepare-data.ipynb

In [ ]:
df = dataframes['eeg'].loc[:, [1,4,7,10]]
df.plot(subplots=True);
# df.columns

## Usage Demo for the MultivariateBasicAUD Sonecule

In [ ]:
from mesonic.synth import Synth
from sonecules.bufferson import MultivariateBasicAUD

We use the realtime mode of the context here.

In [ ]:
ctx.enable_realtime();

The following code cell shows everything needed 
- to create the sonecule with data, 
- to reset the auditory canvas (aka timeline)
- to start the playback at a given rate
- to plot the timeline.

In [ ]:
# create the sonecule from data (e.g. channels 1 and 7 of the EEG data set )
aud = MultivariateBasicAUD.from_df(df, sr=256, columns=[1,7])

# schedule the event (which is just one: to start the synth)
aud.schedule(at=0, rate=20, pan=[-1,1], loop=0, amp=[0.5,0.5]).start()

# plot the data (just for fun)
df.plot(subplots=True);

Note the just heard the audification of channel 1 on the left audio channel and channel 7 on the right audio channel 

Once the MultivariateBasicAUD instance is available, you can replay it as needed with different parameters:

In [ ]:
aud.reschedule(at=0.0, rate=50, pan=0, loop=0, startpos=4000, amp=0.8).start()

Now let's play all channels, spreading them equally between full left and full right to the usual stereo audio interface output.

In [ ]:
ctx.reset()
# create the sonecule from data using all channels by leaving columns unassigned 
aud = MultivariateBasicAUD.from_df(df, sr=256)

# schedule the event (which is just one: to start the synth)
# pan has a special keyword "spread", to distribute all channels from full left to full right
aud.schedule(at=0.0, rate=20, pan="spread", loop=1, amp=0.3).start()

# plot the data (just for fun)
df.plot(subplots=True);

Note that for setting parameters we can now use arrays to affect the channels individually, e.g. to play one channel solo (and mute the others) we can use basis vector arrays as shown next

In [ ]:
aud.set(amp=[1,0,0,0])

In [ ]:
aud.set(amp=[0,0,0,1])

In [ ]:
aud.set(amp=[0.5,0.3,0.3,1])

The parameter `rate` is necessarily a shared parameter

In [ ]:
aud.set(rate=50)

...but stereo position (parameter `pan`) can be adjusted individually!
* let's also set a different startpos, rate and trigger frequency

In [ ]:
aud.set(pan=[-1,1,1,-1], rate=40, trfreq=3, startpos=3000)

In [ ]:
aud.stop()

MultivariateBasicAUD offers the same features as BasicAUD, see the BasicAUD example notebook
- Please check there how to use startpos, trfreq to skim interactively through data

Now we can easily control the Audification with some sliders
* move the startpos slider to skim through the audification
* control rate and trigger rate independently
* the GUI shows howto create faders for the four channels to listen to channels in the foreground with others in the background

In [ ]:
from ipywidgets import interactive
aud.reschedule(at=0, rate=100, pan=[-1,-0.3,0.3,1], loop=1, startpos=0, amp=0.8, trfreq=1).start() 
def aud_gui(startpos=0, trfreq=1, rate=50, amp1=0.1, amp2=0.1, amp3=0.1, amp4=0.1):
    aud.set(startpos=startpos, trfreq=trfreq, rate=rate, amp=[amp1, amp2, amp3, amp4])
interactive(aud_gui, startpos=(0, 12000, 100), trfreq=(1, 50, 1), rate=(1, 200, 1),
    amp1=(0,1,0.01), amp2=(0,1,0.01), amp3=(0,1,0.01), amp4=(0,1,0.01)) 

In [ ]:
# and stop when done
aud.stop()

**Signal Conditioning**

* MultivariateBasicAudification currently doesn't offer filtering or distortion.
* These, however, could be made available in the future or more specialized Sonecules of the AUD family.
* However, some signal conditionings are better applied before audition anyway!
* Modifications such as applying a time scale modification (aka time stretching, i.e. rescaling the time without modifying the spectrum), is for instance well done in pya using Asig.stretch(factor) as shown here for a selected channel and time interval and stretch factor in a one-liner

      aud1 = BasicAUD(my_asig[{1.5:5.2},['channelname']].stretch(3.5))

* so while Sonecules probably don't do it all, combinations with pandas and pya functions enable swift, and flexible implementations of what is needed.

## Code Template

The following code snippets are intended for copy & paste to your notebooks, to facilitate getting your data sonified
using this sonecule.
* It is assumed that your data is stored in an Asig dasig

In [ ]:
# create or load your data, e.g.
df = dataframes['ecg']

# or load data
# df = pd.read_csv("your_csv_file.csv", delimiter=",")
# df = pd.read_excel("your_excel_file.xlsc") # see pandas documenation

data = df.values
column_names = [str(i) for i in df.columns]

# put it into an Asig
a1 = Asig(data, sr=200, cn=column_names)
plt.figure(); a1.plot(offset=1)
a1

In [ ]:
# load your data / select your data
myasig = a1

# sonecule for your synth with defaults and bounds
aud = MultivariateBasicAUD(myasig)

# reset the timeline 
ctx.timeline.reset() 

# finally start the realtime playback at a given rate
aud.schedule(at=0.5, rate=5, loop=1, amp=0.2, pan="spread").start()

# if needed: plot the timeline using 
ctx.timeline.plot()

In [ ]:
# play the 4 outer channels louder
aud.set(amp=[1,-0.3,0,0,-0.3,1], pan=[-1,-1,0,0,1,1], rate=10)

In [ ]:
# play the two center channels and mute the outer
aud.set(amp=[0,0,0.8,0.8,0,0])

In [ ]:
aud.stop()

In [ ]:
ctx.close()  # close the mesonic context, exits backend gracefully